# OpenStreetMap (Data Wrangling with MongoDB)

## Toronto Area
Here are the coordinates I used to analyze an area around Toronto Downtown:  
(node(43.6453,-79.4775,43.7495,-79.3199);<;);out meta;

![title](img/toronto.png)

## Problems Encountered in the Map

During the data wrangling process, I validated and corrected the following values: 

1- Country  
The code validates that the country value for all addresses is "CA". During this validation there was no address found with an invalid value for country field.

2- Province  
The code validates the province value as well. During the audit, I realized that people used different values for province and some of them had typos. So I came up with the list of values that could be replaced by the value that I wanted to have as the province that is "ON". Here is the list of to be replaced values:  
ON: "Ontario", "ontario","On", "on", "Onatrio".  

3- City  
Same for validating the city. The values I wanted to have in my dataset were "Toronto, York, North York, East York". And as for the province, I discovered other possible values people used to name the cities and here is the list of to be replaced values for the cities:  
Toronto: "CityofToronto", "City of Toronto", "Toronto,ON", "Torontoitalian"  
North York: "NorthYork"  
East York: "EastYork"  

4- Postal Code  
The valid postal code format in Canada is [A-Z][0-9][A-Z][ ][0-9][A-Z][0-9]. I audited all the postal code values and I found number of invalid postal codes like A "M36 0H7". I removed these values but I also found that there were a lot of postal codes without the space in between. So I corrected them and I included them in the dataset.  

5- Street  
While auditing the street names, I wanted to remove all the abbreviations. So I searched for any kind of value followed by ".". I found a lot of "St." values in the dataset. I studied more and I figured that this abbreviation in lots of cases is being used for "Saint" and not "Street". So I came up with the list of names (saints) that could come after st. abbreviation and here is the list:  
"Andrew's", "Edmund's", "John's", "Leonard's", "George", "Clair", "Helens", "Dennis", "Joseph", "Clarens", "Matthews", "Hubert", "James", "David", "Thomas", "Mary", "Patrick", "Clements", "Leonards", "Hildas", "Mathias", "Andrews", "Raymond", "Annes", "Cuthberts", "Cuthberts", "Ives", "Edmunds"

In [14]:
import xml.etree.cElementTree as ET
import pprint
import re
import codecs
import json
from collections import defaultdict

#find the two-level values 
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')

#find the three-level values 
lower_double_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*:([a-z]|_)*$')

#find the problamatic values 
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

#valid country  
canada = re.compile(r'^(CA)$')

#valid province
ontario = re.compile(r'^(ON)$')

#other province values to be converted to "ON"
ontario_alternative = ["Ontario", "ontario","On", "on", "Onatrio"]

#valid cities
city = re.compile(r'^(Toronto|York|North York|East York)$')

#other toronto values to be converted to "Toronto"
toronto_alternative = ["CityofToronto", "City of Toronto", "Toronto,ON", "Torontoitalian"]

#other North York values to be converted to "North York"
northyork_alternative = ["NorthYork"]

#other East York values to be converted to "East York"
eastyork_alternative = ["EastYork"]

#valid postal code format
postcode = re.compile(r'^([A-Z][0-9][A-Z][ ][0-9][A-Z][0-9])$')

#other postal code format to be converted to the valid one
postcode_alternative = re.compile(r'^([A-Z][0-9][A-Z][0-9][A-Z][0-9])$')

#any type of abbreviation
street = re.compile(r'\b([a-z]|_)*\.', re.IGNORECASE)

#Created object 
CREATED = [ "version", "changeset", "timestamp", "user", "uid"]

#Position object
POSITION = [ "lat", "lon"]

#valid names to come after St.
SAINTS = ["Andrew's", "Edmund's", "John's", "Leonard's", "George", "Clair", "Helens", "Dennis", "Joseph", "Clarens",
          "Matthews", "Hubert", "James", "David", "Thomas", "Mary", "Patrick", "Clements", "Leonards", "Hildas", "Mathias",
          "Andrews", "Raymond", "Annes", "Cuthberts", "Cuthberts", "Ives", "Edmunds"]

#dictionary of all problamtic items by category
problematic_elements = {}
problematic_elements["key_invalid_char"] = []
problematic_elements["key_double_colon"] = []
problematic_elements["country"] = []
problematic_elements["province"] = []
problematic_elements["city"] = []
problematic_elements["street"] = []
problematic_elements["postcode"] = []

#create json object based on element
def shape_element(element):
    node = {}
    if element.tag == "node" or element.tag == "way" :
        node["id"] = element.attrib["id"]
        node["type"] = element.tag
        for key,value in element.attrib.iteritems():
            if key in CREATED:
                if 'created' not in node:
                    node['created'] = {}
                node["created"][key] = value
            elif key in POSITION:
                if 'pos' not in node:
                    node['pos'] = [None]*2
                if key == 'lat':
                    node["pos"][0] = float(value)
                elif key == 'lon':
                    node["pos"][1] = float(value)
            else:
                node[key] = value
            
        for tag in element.iter():
            if tag.tag == 'tag':
                tag_key = tag.attrib["k"]
                if is_match(problemchars, tag_key):
                    problematic_elements["key_invalid_char"].append(tag_key)
                    continue
                if is_match(lower_double_colon , tag_key):
                    problematic_elements["key_double_colon"].append(tag_key)
                    continue
                if is_match(lower_colon , tag_key):
                    parse_colon(node, tag, tag_key)
                else:
                    is_valid_value,final_value = check_value(tag_key, tag.attrib['v'])
                    if is_valid_value:
                        node[tag_key] = final_value
            elif tag.tag == 'nd':
                if 'node_refs' not in node:
                    node['node_refs'] = []
                node['node_refs'].append(tag.attrib['ref'])
       
        return node
    else:
        return None

# create two level objects like address
def parse_colon(node, element, tag_key):
    key_part_1,key_part_2 = tag_key.split(':')
    if key_part_1 not in node:
        node[key_part_1] = {}
    elif type(node[key_part_1]) is not dict:
        type_name = node[key_part_1]
        node[key_part_1] = {}
        node[key_part_1]["type"] = type_name
    is_valid_value,final_value = check_value(tag_key, element.attrib['v'])
    if is_valid_value:
        node[key_part_1][key_part_2] = final_value

# validate and convert value based on key. it return a tuple of the following
# boolean : indicating whether the validation was successful
# string : original or converted value, Empty string if validation fails 
def check_value(key, value): 
    if key == 'addr:country':
        m = canada.search(value)
        if m:
            return True,value
        problematic_elements["country"].append(value)
        return False,""
    if key == 'addr:province':
        m = ontario.search(value)
        if m:
            return True,value
        elif value in ontario_alternative:
            return True,"ON"
        problematic_elements["province"].append(value)
        return False,""
    if key == 'addr:city':        
        m = city.search(value)
        if m:
            return True,value
        elif value in toronto_alternative:
            return True,"Toronto"
        elif value in northyork_alternative:
            return True,"North York"
        elif value in eastyork_alternative:
            return True,"East York"
        problematic_elements["city"].append(value)
        return False,""
    if key == 'addr:postcode':        
        m = postcode.search(value)
        if m:
            return True,value
        ma = postcode_alternative.search(value)
        if ma:
            return True,value[:3]+" "+value[3:]
        problematic_elements["postcode"].append(value)
        return False,""
    if key == 'addr:street':        
        m = street.search(value)
        if m:
            saint_name = value[4:].split(' ')
            if value[0:4] == 'St. ' and saint_name[0] in SAINTS:
                return True, "Saint " + value[4:]
            problematic_elements["street"].append(value)
            return False,""
    
    return True,value

#match a regular expression with an item
def is_match(rg,item):
    m = rg.search(item)
    if m:
        return True
    return False

#main function
def process_map(file_in, pretty = False):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

data = process_map('toronto.osm', True)

## Data Overview
Here is the overview statistics of the dataset

toronto.osm      105,444KB  
toronto.osm.json 146,656KB  

>db.toronto.find().count()  
474579  

>db.toronto.find({"type":"node"}).count()  
396867  

>db.toronto.find({"type":"way"}).count()  
77589  



### Other queries
 
Total number of street names with the word "Saint" in it:  
>db.toronto.find({"addr.street": /Saint/}).count()  
1068

Top 10 contributing users:  
>db.toronto.aggregate([{"\$group":{"_id" : "\$created.user","count" : {"\$sum":1}}}, {"\$sort" : {"count":-1}},{"\$limit":10}])  
{ "_id" : "andrewpmk", "count" : 360571 }  
{ "_id" : "Kevo", "count" : 36098 }  
{ "_id" : "Mojgan Jadidi", "count" : 14407 }  
{ "_id" : "andrewpmk_imports", "count" : 6254 }  
{ "_id" : "TristanA", "count" : 5451 }  
{ "_id" : "Nate_Wessel", "count" : 4953 }  
{ "_id" : "Shrinks99", "count" : 2748 }  
{ "_id" : "Nate_Wessel (consulting)", "count" : 2514 }  
{ "_id" : "lorandr_telenav", "count" : 2423 }  
{ "_id" : "Bootprint", "count" : 2138 }  

Top 10 banks in the area:  
>db.toronto.aggregate([{"\$match":{"amenity":{"\$exists":1}, "amenity":"bank"}}, {"\$group":{"_id":"\$name", "count":{"\$sum":1}}},{"\$sort":{"count":-1}}, {"\$limit":10}])  
{ "_id" : "TD Canada Trust", "count" : 60 }  
{ "_id" : "Scotiabank", "count" : 43 }  
{ "_id" : "CIBC", "count" : 34 }  
{ "_id" : "BMO Bank of Montreal", "count" : 29 }  
{ "_id" : "RBC", "count" : 14 }  
{ "_id" : "RBC Royal Bank", "count" : 14 }  
{ "_id" : "RBC Financial Group", "count" : 11 }  
{ "_id" : "CIBC Banking Centre", "count" : 10 }  
{ "_id" : "Bank of Montreal", "count" : 9 }  
{ "_id" : "HSBC", "count" : 7 }  

Total number of address in each city
>db.toronto.aggregate([{"\$group":{"_id":"\$addr.city","count":{"\$sum":1}}}])  
{ "_id" : "York", "count" : 239 }  
{ "_id" : "North York", "count" : 942 }  
{ "_id" : "Toronto", "count" : 41544 }  
{ "_id" : "East York", "count" : 750 }  
{ "_id" : null, "count" : 431104 }  

Top 10 cuisines in the area
>db.toronto.aggregate([{"\$match":{"amenity":{"\$exists":1}, "amenity":"restaurant"}}, {"\$group":{"_id":"\$cuisine", "count":{"\$sum":1}}},{"\$sort":{"count":-1}}, {"\$limit":10}])  
{ "_id" : null, "count" : 571 }  
{ "_id" : "japanese", "count" : 38 }  
{ "_id" : "italian", "count" : 27 }  
{ "_id" : "indian", "count" : 26 }  
{ "_id" : "chinese", "count" : 22 }  
{ "_id" : "sushi", "count" : 19 }  
{ "_id" : "thai", "count" : 17 }  
{ "_id" : "asian", "count" : 14 }  
{ "_id" : "mexican", "count" : 13 }  
{ "_id" : "pizza", "count" : 12 }  



## Additional Ideas
As described before, I kept the problematic entries by their category in the problematic_elements dictionary. Looking at the problematic province and city entries before replacing the faulty values, I had 168 invalid province entry and 34081 invalid city entry.  
>print len(problematic_elements["province"])  
print len(problematic_elements["city"])  
168  
34081  

Thinking about that, and I noticed that these kind of inconsistency could have been prevented during the data entry. For example, OpenStreetMap could detect that the user is entering data for Toronto region and give user an option to reuse predefined values for country, province and city. The benefit of proving this kind of predefined value is that it decreases the chance of human error during the data entry and also it could eventually define a standard way of contributing to OpenStreetMap for different entities like address. The downside however is that predefined values themselves would need maintenance. They might not be complete or there might be inconsistency or even error in them. So we will need another process to audit these values.